# ACR extraction #

Parse text documents which are based on a template and stored in an excel document

Install dependencies and import the nlp_datau package

In [ ]:
import yaml
import re
import sys

!pip install -r ./../requirements.txt
sys.path.append("./../")


## 1. Extract substring of substring ##

Set variables

In [ ]:
path_excel_sheet_in = '/home/jovyan/work/D/ACR-meta/Mammo-20211011-run1.xlsx'
# path_excel_sheet_in = '/home/jovyan/work/resources/data_example/test_acr.xlsx'
path_excel_sheet_in = '/home/jovyan/work/resources/data_ignored/ACR/Mammo-20211011-run1.xlsx'

# path_excel_sheet_out = '/home/jovyan/work/resources/data_example/test_acr.xlsx'
path_excel_sheet_out = '/home/jovyan/work//resources/data_ignored/ACR/Mammo-20211011-run1-out.xlsx'




import pandas
from IPython.core.display import display
df = pandas.read_excel(path_excel_sheet_in, header=0)
display(df)

Extract findings

In [ ]:
column_source = 'text'

column_names_list = [
    'acr3',
    'acr4',
    'bevolkingsonderzoek'
]

# https://regex101.com, eerste / kan weg

regex_list = [
    r"(hetrogeen dens.{0,6}klier|ACR( |-| klasse )(3|drie|III|c|C)|hetrogeen.{0,5}trial)",
    r"(zeer dens.{0,6}klier|ACR( |-| klasse )(4|vier|IV|d|D)|dens.{0,5}trial)",
    r"(doorver|BVO|bevolking.{0,2}onderzoek)"
]

In [ ]:
flags = re.IGNORECASE
def has_match(search_regex, txt):
    if not txt or not isinstance(txt, str):
        return False
    return bool(re.search(search_regex, txt, flags=flags))

print(has_match(regex_list[0], "ACR-3"))
print(has_match(regex_list[0], "ACR-4"))
print(has_match(regex_list[1], "ACR-3"))
print(has_match(regex_list[1], "ACR-4"))

Extract corads

In [ ]:
for index, column_target in enumerate(column_names_list):
    search_regex = regex_list[index]
    df[column_target] = df.apply(lambda x:has_match(search_regex=search_regex, txt=x[column_source]), axis=1)

display(df)

Run and write to out file

In [ ]:
for col in column_names_list:
    print(df[col].value_counts())

In [ ]:
df.to_excel(path_excel_sheet_out)

In [ ]:
df['acr3+bev'] = df.acr3 * df.bevolkingsonderzoek

In [ ]:
df['acr4+bev'] = df.acr4 * df.bevolkingsonderzoek

In [ ]:
df['acr4+bev'].value_counts()

In [ ]:
df['acr3+bev'].value_counts()

In [ ]:
df['acr+bev'] = df['acr3+bev'] | df['acr4+bev']

In [ ]:
df['acr+bev'].value_counts()

In [ ]:
df.to_excel(path_excel_sheet_out)